In [70]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.cm as cm
import matplotlib as mpl
import xesmf as xe
from workflow.scripts.utils import regrid_global
import numpy as np
from pyclim_noresm.general_util_funcs import global_avg

In [64]:
def read_data(paths):
    dsets = []
    for p in paths:
        ds = xr.open_dataset(p)
        ds = ds.cf.add_bounds(['lon','lat'])
        grid_params = snakemake.config['regrid_params']
        method=grid_params.get('method','conservative')

        dxdy = grid_params['dxdy']
        ds = regrid_global(ds, lon=dxdy[0], lat=dxdy[1], method=method)
        da = ds[ds.variable_id]
#         return da
        da = da.rename(f'{ds.variable_id}_{ds.source_id}')
        da = da.assign_coords(year=np.arange(0, len(da.year)))
        da = da.rename(year='time')
        dsets.append(da)
    out_da = xr.merge(dsets)
    return out_da

In [65]:
ds2xdust = read_data(snakemake.input.piCli2xdust)
dsaer = read_data(snakemake.input.piCliaer)

In [68]:
piClim_aer_var = dsaer- dsaer.mean(dim='time')

In [69]:
piClim_dust_var = ds2xdust - ds2xdust.mean(dim='time')

In [77]:
df_piClimaer =global_avg(piClim_aer_var).to_pandas()
df_piClim2xdust = global_avg(piClim_dust_var).to_pandas()

In [87]:
fig,ax = plt.subplots(ncols=2,figsize=(14,6), sharey=True)

ax[0].axhline(0, color='k')
df_piClimaer.plot(ax=ax[0])
ax[0].set_ylim(-1,1)
ax[1].axhline(0, color='k')
df_piClim2xdust.plot(ax=ax[1])
ax[1].set_ylim(-1,1)
ax[0].set_ylabel('W/m^2')
plt.savefig(snakemake.output.outpath, bbox_inches=)

In [92]:
df_piClimaer.std()

In [91]:
df_piClim2xdust.std()

In [88]:
df_aer = global_avg(dsaer).to_pandas()
df_dust = global_avg(ds2xdust).to_pandas()

In [90]:
fig,ax = plt.subplots(ncols=2,figsize=(14,6), sharey=True)

ax[0].axhline(0, color='k')
df_aer.plot(ax=ax[0])
# ax[0].set_ylim(-1,1)
ax[1].axhline(0, color='k')
df_dust.plot(ax=ax[1])
# ax[1].set_ylim(-1,1)
ax[0].set_ylabel('W/m^2')